# Compare lithium-ion battery models

We compare three one-dimensional lithium-ion battery models: [the Doyle-Fuller-Newman (DFN) model](./DFN.ipynb), [the single particle model (SPM)](./SPM.ipynb), and [the single particle model with electrolyte (SPMe)](./SPMe.ipynb). Further details on these models can be found in [[4]](#References).


## Key steps:

Comparing models consists of 6 easy steps:

 1. Load models and geometry
 2. Process parameters
 3. Mesh the geometry
 4. Discretise models 
 5. Solve models
 6. Plot results 

But, as always we first import pybamm and other required modules

In [ ]:
import pybamm
import os

os.chdir(pybamm.__path__[0] + "/..")

import numpy as np
import matplotlib.pyplot as plt

## 1. Load models

Since the three models we want to compare are already implemented in PyBaMM, they can be easy loaded using: 

In [ ]:
dfn = pybamm.lithium_ion.DFN()
spme = pybamm.lithium_ion.SPMe()
spm = pybamm.lithium_ion.SPM()

To allow us to perform the same operations on each model easily, we create a dictionary of these three models:

In [ ]:
models = {"DFN": dfn, "SPM": spm, "SPMe": spme}

Each model can then be accessed using:

In [ ]:
models["DFN"]

For each model, we must also provide a cell geometry. The geometry is different for different models; for example, the SPM has solves for a single particle in each electrode whereas the DFN solves for many particles. For simplicity, we use the default geometry associated with each model but note that this can be easily changed.

In [ ]:
geometry = {
    "DFN": dfn.default_geometry,
    "SPM": spm.default_geometry,
    "SPMe": spme.default_geometry,
}

## 2. Process parameters

For simplicity, we use the default parameters values associated with the DFN model, but change the current function to be an input so that we can quickly solve the model with different currents

In [ ]:
param = dfn.default_parameter_values
param["Current function [A]"] = "[input]"

It is simple to change this to a different parameter set if desired. 

We then process the parameters in each of the models and geometries using this parameter set:

In [ ]:
for model_name in models.keys():
    param.process_model(models[model_name])
    param.process_geometry(geometry[model_name])

## 3. Mesh geometry

We use the defaults mesh properties (the types of meshes and number of points to be used) for simplicity to generate a mesh of each model geometry. We store these meshes in a dictionary of similar structure to the geometry and models dictionaries:

In [ ]:
mesh = {}
for model_name, model in models.items():
    mesh[model_name] = pybamm.Mesh(
        geometry[model_name], model.default_submesh_types, model.default_var_pts
    )

## 4. Discretise model

We now discretise each model using its associated mesh and the default spatial method associated with the model:

In [ ]:
for model_name, model in models.items():
    disc = pybamm.Discretisation(mesh[model_name], model.default_spatial_methods)
    disc.process_model(model)

## 5. Solve model

We now solve each model using the default solver associated with each model:

In [ ]:
timer = pybamm.Timer()
solutions = {}
t_eval = np.linspace(0, 3600, 300)  # time in seconds
for model_name, model in models.items():
    solver = pybamm.CasadiSolver()
    timer.reset()
    solution = solver.solve(model, t_eval, inputs={"Current function [A]": 1})
    print(f"Solved the {model.name} in {timer.time()}")
    solutions[model_name] = solution

## 6. Plot results

To plot results, we extract the variables from the solutions dictionary. Matplotlib can then be used to plot the voltage predictions of each models as follows:

In [ ]:
for model_name, model in models.items():
    time = solutions[model_name]["Time [s]"].entries
    voltage = solutions[model_name]["Voltage [V]"].entries
    plt.plot(time, voltage, lw=2, label=model.name)
plt.xlabel("Time [s]", fontsize=15)
plt.ylabel("Voltage [V]", fontsize=15)
plt.legend(fontsize=15)
plt.show()

Alternatively the inbuilt `QuickPlot` functionality can be employed to compare a set of variables over the discharge. We must first create a list of the solutions

In [ ]:
list_of_solutions = list(solutions.values())

And then employ `QuickPlot`:

In [ ]:
quick_plot = pybamm.QuickPlot(list_of_solutions)
quick_plot.dynamic_plot();

# Changing parameters

Since we have made current an input, it is easy to change it and then perform the calculations again:

In [ ]:
# update parameter values and solve again
# simulate for shorter time
t_eval = np.linspace(0, 800, 300)
for model_name, model in models.items():
    solutions[model_name] = model.default_solver.solve(
        model, t_eval, inputs={"Current function [A]": 3}
    )

# Plot
list_of_solutions = list(solutions.values())
quick_plot = pybamm.QuickPlot(list_of_solutions)
quick_plot.dynamic_plot();

By increasing the current we observe less agreement between the models, as expected. 

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()